In [2]:
import numpy as np
from nltk.tokenize import RegexpTokenizer
from keras.models import Sequential, load_model
from keras.layers import LSTM
from keras.layers.core import Dense, Activation
from keras.optimizers import RMSprop
import heapq
import pickle


# usando o arquivo '1661-0.txt' que é muito utilizado para desenvolvimento de uma LSTM
path = '1661-0.txt'
text = open(path).read().lower()
print('quantidade de palavras: ', len(text))

quantidade de palavras:  581533


In [3]:
# Tokenização para apenas os caracteres alfanuméricos serem considerados como parte dos tokens 
# E cada token ser uma palavra do texto original guardada em word
tokenizer = RegexpTokenizer(r'\w+')
words = tokenizer.tokenize(text)

# Criação do dicionário de palavras únicas
# O NumPy é usado para realizar essa operação de forma eficiente

unique_words = np.unique(words)
unique_word_index = dict((c, i) for i, c in enumerate(unique_words))

# Essa variável representa o número de palavras consecutivas que serão usadas como entrada
WORD_LENGTH = 5

# Essas variáveis são usadas em um loop posterior para criar as sequências de palavras anteriores e suas respectivas palavras seguintes.
prev_words = []
next_words = []

# Geração dos pares de palavras de entrada e saída

for i in range(len(words) - WORD_LENGTH):
    prev_words.append(words[i:i + WORD_LENGTH])
    next_words.append(words[i + WORD_LENGTH])

print(prev_words[0], prev_words[1], prev_words[2])
print(next_words[0], next_words[1])


['the', 'project', 'gutenberg', 'ebook', 'of'] ['project', 'gutenberg', 'ebook', 'of', 'the'] ['gutenberg', 'ebook', 'of', 'the', 'adventures']
the adventures


# As palavras "the" e "adventures" são a sequência do primeiro bloco de palavras consecutivas, ou seja, a divisao prev_word e next_word estão corretas


In [4]:
# criamos matrizes vazias X e Y
# onde X será usada para armazenar a codificação one-hot das sequências de palavras anteriores 
# Y será usada para armazenar a codificação one-hot das palavras seguintes correspondentes.
X = np.zeros((len(prev_words), WORD_LENGTH, len(unique_words)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_words)), dtype=bool)

# preencher as matrizes X e Y 
# com os valores da codificação one-hot das sequências de palavras anteriores e suas respectivas palavras seguintes
for i, each_words in enumerate(prev_words):
    for j, each_word in enumerate(each_words):
        X[i, j, unique_word_index[each_word]] = 1
    Y[i, unique_word_index[next_words[i]]] = 1


In [5]:
# Definição e compilação do modelo LSTM
model = Sequential()
model.add(LSTM(128, input_shape=(WORD_LENGTH, len(unique_words))))
model.add(Dense(len(unique_words)))
model.add(Activation('softmax'))

# definir o otimizador RMSprop como o otimizador para o modelo.
optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Treinamento do modelo
history = model.fit(X, Y, validation_split=0.05, batch_size=128, epochs=10, shuffle=True)



/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/rmsprop.py:143: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/10
811/811 [==============================] - 19s 15ms/step - loss: 6.0153 - accuracy: 0.1075 - val_loss: 6.9843 - val_accuracy: 0.1066
Epoch 2/10
811/811 [==============================] - 12s 14ms/step - loss: 5.7772 - accuracy: 0.1483 - val_loss: 7.8278 - val_accuracy: 0.1073
Epoch 3/10
811/811 [==============================] - 12s 14ms/step - loss: 5.7539 - accuracy: 0.1766 - val_loss: 7.7848 - val_accuracy: 0.1068
Epoch 4/10
811/811 [==============================] - 11s 14ms/step - loss: 5.4333 - accuracy: 0.2117 - val_loss: 8.1528 - val_accuracy: 0.0965
Epoch 5/10
811/811 [==============================] - 12s 14ms/step - loss: 5.0987 - accuracy: 0.2504 - val_loss: 8.1385 - val_accuracy: 0.1013
Epoch 6/10
811/811 [==============================] - 11s 14ms/step - loss: 4.8145 - accuracy: 0.2900 - val_loss: 8.4423 - val_accuracy: 0.0973
Epoch 7/10
811/811 [==============================] - 12s 14ms/step - loss: 4.5610 - accuracy: 0.3286 - val_loss: 8.5966 - val_accuracy:

# Mesmo aumentando o batch size, o custo computacional fica sendo muito alto por epoch. Isso faz com que o treinamento seja curto e o valor da accuracy seja baixo, porém observando o valor é nitido a melhora a cada epoch, ou seja, com um recurso computacional melhor, teríamos um resultado bem melhor também na predição

In [12]:

# Função para preparar a entrada do modelo e prever as próximas palavras
def predict_completions(text, n=3):
    x = np.zeros((1, WORD_LENGTH, len(unique_words)))
    for t, word in enumerate(text.split()):
        x[0, t, unique_word_index[word]] = 1

    preds = model.predict(x, verbose=0)[0]
    
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds)
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)

    next_indices = heapq.nlargest(n, range(len(preds)), preds.take)
    return [unique_words[idx] for idx in next_indices]

# Teste de predição de palavras
setence = "Your life will never be the same again"
print("Correct sentence:", setence)
sequencia = " ".join(tokenizer.tokenize(setence.lower())[0:4])
print("Sequence:", sequencia)
print("Next possible words:", predict_completions(sequencia, 10))

Correct sentence: Your life will never be the same again
Sequence: your life will never
Next possible words: ['your', 'him', 'a', 'go', 'before', 'be', 'say', 'my', 'no', 'find']
